In [ ]:
import json 
from pat_back import PatJob, AppSettings

# Submit Job

In [ ]:
job_para = json.load(open('data/pat_job_test.json'))
job_para

# Run Sequential Code 

In [ ]:
%%time

job = PatJob(job_para) 
if job.job_id> 0:
    job.perform_analysis()    

ret = PatJob.get_results(job.job_id)
res = None
if ret is not None:
    res = pd.DataFrame(json.loads(ret))


# Run with Service

In [ ]:
import requests
import uuid
import json

# files = {'para': ('pat_para', open('data/pat_job_test.json','rt'))}
job_para = json.load(open('data/pat_job_test.json'))
job_para['job_guid'] = str(uuid.uuid4())
job_para['job_name'] = 'AIG Test no correction_t2'

# files = {
#         'para': json.dumps(job_para),
#         # 'data': open(r'C:\_Working\PAT_20201019\debug\pat.valid.zip','rb')
#     }

# ret = requests.post("http://frkcat-ws10:8002/api/Jobs", files=files)
ret = requests.post("http://frkcat-ws10:8002/api/Jobs", json=job_para)
# ret = requests.post("http://localhost:5000/api/Jobs", json=job_para)
ret.status_code, ret.content

# Batch submit

In [ ]:
import requests
import uuid
import json

job_para = {
  "server": "FRKCAT-RMS4SQL1",
  "edm": "Chubb_CLNA_BUILD_20210701_EDM_v181",
  "rdm": "Chubb_CLNA_2021_FireGELwPol_RDMv181",
  "portinfoid": 7,
  "perilid": 2,
  "analysisid": 31,
  "type_of_rating": "PSOLD",
  "peril_subline": "All Perils",
  "coverage": "Building + Contents + Time Element",
  "loss_alae_ratio": 1,
  "average_accident_date": "10/1/2022",
  "trend_factor": 1.035,
  "additional_coverage": 2,
  "deductible_treatment": "Retains Limit",
  "valid_rules": -1,
  "job_name": "Chubb_2021_MedRisk",
  "user_name": "Chengyou",
  "user_email": "chengyou.xiao@guycarp.com",
  "job_guid": "2064e7f3-2292-4944-aae6-7b8d23ef3507"
}

lst=[('NACI',8,5,9),
('MedRisk',7,2,31),
('Advantage',2,5,3),
('CanadaCombined',4,5,5),
('FARM_HUREQ',5,2,29),
('WSGBF',14,5,13),
('WSGIM',15,5,15),
('SCI',13,5,11),
('WSGPD',16,5,17),
('CM_Warehouse',18,5,21),
('OMC',17,5,19),
('CM_IM',19,5,23),
('Programs',30,5,27),
('MAP',6,5,7),
('BDA',29,5,25),
('ACEIM',1,5,2)]

for port in lst:
    job_para['job_guid'] = str(uuid.uuid4())
    job_para['job_name'] = f'Chubb_2021_{port[0]}'
    job_para['portinfoid'] = port[1]
    job_para['perilid'] = port[2]
    job_para['analysisid'] = port[3]

    ret = requests.post("http://frkcat-ws10:8002/api/Jobs", json=job_para)
    print(port[0],ret.status_code, ret.content)


# Results Summary

In [ ]:
import requests
import pandas as pd

job_id = 100000196
actions = {
    'list': 'api/Jobs',
    'para': f'api/Jobs/{job_id}/Para',
    'summary': f'api/Jobs/{job_id}',
    'result': f'api/Jobs/{job_id}/Result',
    'validation': f'api/Jobs/{job_id}/Validation',
}

ret = requests.get(f"http://frkcat-ws10:8002/{actions['list']}")
try:
    df = pd.DataFrame(ret.json())
except:
    df = None
    print(ret.content)

df
# print(json.dumps(ret.json(), indent=4))

In [6]:
import requests

job_id  = 100000075
requests.put(f"http://localhost:5000/api/public-job/{job_id}")

<Response [200]>

In [ ]:
df.groupby(by='LocationIDStack').agg({'Premium':'sum'}) 

In [ ]:
import requests

requests.post(f"http://localhost:5000/api/reset/1_2_0_3")

In [ ]:
import requests
import uuid
import json

job_para = {
    "additional_coverage": 2,
    "analysisid": 9,
    "average_accident_date": "10/1/2022",
    "coverage": "Building_Contents_BI",
    "deductible_treatment": "Retains_Limit",
    "edm": "Chubb_CLNA_BUILD_20210701_EDM_v181",
    "valid_rules": -1,
    "job_guid": "755f191e-ace9-4491-afb4-34afc6e4a10b",
    "job_name": "Chubb_2021_NACI_try",
    "loss_alae_ratio": 1,
    "peril_subline": "All_Perils",
    "perilid": 5,
    "portinfoid": 8,
    "rdm": "Chubb_CLNA_2021_FireGELwPol_RDMv181",
    "server": "FRKCAT-RMS4SQL1",
    "subject_premium": 1,
    "trend_factor": 1.035,
    "type_of_rating": "PSOLD_2016",
    "user_email": "chengyou.xiao@guycarp.com",
    "user_name": "Chengyou"
}

ret = requests.post("http://localhost:5000/api/job", json = job_para)
ret.status_code, ret.content

In [19]:
import numpy as np

timeit()
1000 * np.power(10, np.arange(11) * 0.5)

array([1.00000000e+03, 3.16227766e+03, 1.00000000e+04, 3.16227766e+04,
       1.00000000e+05, 3.16227766e+05, 1.00000000e+06, 3.16227766e+06,
       1.00000000e+07, 3.16227766e+07, 1.00000000e+08])

In [7]:
import json
import numpy as np
import pandas as pd
import pyodbc

conn_str = f'''DRIVER={{SQL Server}};Server=dfwcat-pm;Database=premium_allocation_tool;
            User Id=sa;Password=In5tr@t1;MultipleActiveResultSets=true;'''

def transform(row):
    job_id = row[0]
    js= json.loads(row[1])

    if 'psold' in js:
        if 'blending_weights' not in js['psold']:
            wlst= np.array(js['psold']['blending']) if 'blending' in js['psold'] else None
            if wlst is not None and np.any(wlst>0):
                js['psold']['blending_type'] = 'missing_invalid'
                js['psold']['blending_weights'] = js['psold']['blending']
                if 'hpr_blending' not in js['psold']: js['psold']['hpr_blending'] = False
            else:
                js['psold']['blending_type'] = 'no_blending'
                if 'hpr_blending' in js['psold']: del js['psold']['hpr_blending']
            
            if 'blending' in js['psold']: del js['psold']['blending']
            if "average_accident_date" in js:
                js['psold']['average_accident_date'] = js['average_accident_date']
                del js['average_accident_date']

    if "average_accident_date" in js: del js['average_accident_date']

    with pyodbc.connect(conn_str) as conn, conn.cursor() as cur:
        cur.execute(f"""update pat_job set parameters = '{json.dumps(js).replace("'", "''")}'
                where job_id = {job_id}""")
        cur.commit()

    print(job_id)    

with pyodbc.connect(conn_str) as conn:
    df = pd.read_sql_query("""select job_id, parameters from pat_job order by job_id""", conn)
    
df.apply(transform, axis =1)

100000000
100000010
100000011
100000012
100000013
100000014
100000015
100000016
100000017
100000018
100000019
100000020
100000021
100000022
100000023
100000024
100000025
100000026
100000027
100000028
100000029
100000030
100000031
100000032
100000033
100000034
100000035
100000036
100000037
100000038
100000039
100000040
100000041
100000042
100000043
100000044
100000045
100000046
100000047
100000048
100000049
100000050
100000051
100000052
100000053
100000054
100000055
100000056
100000057
100000058
100000059
100000060
100000061
100000062
100000064
100000065
100000066
100000067
100000068
100000069
100000070
100000071
100000072
100000073
100000074
100000075
100000076
100000077
100000078
100000079
100000080
100000081
100000082
100000083
100000101
100000102
100000119
100000126
100000132
100000134


0     None
1     None
2     None
3     None
4     None
      ... 
75    None
76    None
77    None
78    None
79    None
Length: 80, dtype: object

In [11]:
import requests

rg='_'.join([f'{a}' for a in [100000081, 100000083]])

requests.delete(f"http://localhost:5000/api/job/{rg}")



<Response [200]>

In [10]:
import requests	
requests.put(f"http://localhost:5000/api/public-job/100000102")    

<Response [200]>